In [ ]:
from abc import ABCMeta
from functools import reduce
import threading
import multiprocessing as mp
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator
from sklearn.metrics import mean_squared_error
from sklearn.cluster import KMeans
from scipy import stats
from ast import literal_eval
import datetime
from datetime import datetime, timedelta
import time
import pytz
import json
import math
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns
from tick.plot import plot_hawkes_kernels, plot_point_process
from tick.hawkes import HawkesExpKern, HawkesADM4, SimuHawkesExpKernels

In [ ]:
dataset = pd.read_csv(str(input()))
dataset

In [ ]:
#dataset.drop(['D'], axis=1, inplace = True)
#dataset
len(dataset['course_code'].unique())

In [ ]:
len(dataset['metadata_context_id'].unique())

In [ ]:
len(dataset['metadata_user_id'].unique())

In [ ]:
timestampColumns = ['A', 'C', 'L']

In [ ]:
startDate = datetime.strptime(str(input("Start Date (Format: YYYY-MM-DD): ")), "%Y-%m-%d").astimezone(pytz.utc)
startDate

In [ ]:
endDate = datetime.strptime(str(input("End Date (Format: YYYY-MM-DD): ")), "%Y-%m-%d").astimezone(pytz.utc)
endDate

In [ ]:
dayAfterEndDate = float((time.mktime(endDate.timetuple()) / 3600) - (time.mktime(startDate.timetuple()) / 3600))
dayAfterEndDate

In [ ]:
def calculateActualIntensities(timestampList):
    result = {}
    for timePoint in timestampList:
        if math.floor(timePoint) not in result.keys():
            result[math.floor(timePoint)] = 0
        result[math.floor(timePoint)] = result[math.floor(timePoint)] + 1
    
    return result

for column in timestampColumns:
    dataset[column] = dataset[column].apply(lambda x: literal_eval(x))
    dataset[column] = dataset[column].apply(sorted)
    dataset['actual_intensities_' + column] = dataset[column].apply(calculateActualIntensities)

dataset

In [ ]:
def combineColumns(df, columns):
    return [np.array(df[column], dtype=np.double) for column in columns]

dataset['timestamps'] = dataset.apply(lambda df: combineColumns(df, timestampColumns), axis = 1)
dataset['timestamps_train'] = dataset['timestamps']
dataset

In [ ]:
def estimateBestDecay(timestamps):
    highestScore = None
    bestDecay = None
    for d in range(1, 11):
        learner = HawkesExpKern(decays=d, penalty='elasticnet', elastic_net_ratio=0.5)
        learner.fit(timestamps)
        score = learner.score(timestamps, end_times=dayAfterEndDate)
        if((highestScore == None) or (score >= highestScore)):
            highestScore = score
            bestDecay = d
    
    return bestDecay

In [ ]:
try:
    decays = pd.read_csv(str(input()))
    dataset = dataset.merge(decays, on=['metadata_context_id', 'metadata_user_id'])
except:
    dataset['decay'] = dataset['timestamps_train'].apply(estimateBestDecay)
    dataset[['metadata_context_id', 'metadata_user_id', 'decay']].to_csv('MultivariateHawkesDecays_Intercession_2022.csv', header = True, index = False)

dataset

In [ ]:
def getHawkesProcessExpKernAdjacency(df):
    learner = HawkesExpKern(decays=df['decay'], penalty='elasticnet', elastic_net_ratio=0.5)
    learner.fit(df['timestamps_train'])
    return learner.adjacency

dataset['adjacency'] = dataset.apply(getHawkesProcessExpKernAdjacency, axis = 1)
dataset

In [ ]:
def getHawkesProcessExpKernBaseline(df):
    learner = HawkesExpKern(decays=df['decay'], penalty='elasticnet', elastic_net_ratio=0.5)
    learner.fit(df['timestamps_train'])
    return learner.baseline

dataset['baseline'] = dataset.apply(getHawkesProcessExpKernBaseline, axis = 1)
dataset

In [ ]:
def getHawkesProcessExpKernIntensities(df):
    learner = HawkesExpKern(decays=df['decay'], penalty='elasticnet', elastic_net_ratio=0.5)
    learner.fit(df['timestamps_train'])
    intensities, intensity_times = learner.estimated_intensity(df['timestamps'], intensity_track_step=1, end_time=dayAfterEndDate)
    return intensities

dataset['intensities'] = dataset.apply(getHawkesProcessExpKernIntensities, axis = 1)
dataset

In [ ]:
def getHawkesProcessExpKernIntensityTimes(df):
    learner = HawkesExpKern(decays=df['decay'], penalty='elasticnet', elastic_net_ratio=0.5)
    learner.fit(df['timestamps_train'])
    intensities, intensity_times = learner.estimated_intensity(df['timestamps'], intensity_track_step=1, end_time=dayAfterEndDate)
    return intensity_times

dataset['intensity_times'] = dataset.apply(getHawkesProcessExpKernIntensityTimes, axis = 1)
dataset

In [ ]:
def splitResultsExpKern(overallResult, dimensions):
    for i in range(0, len(dimensions)):
        overallResult['baseline_' + dimensions[i]] = overallResult['baseline'].apply(lambda b: b.tolist()[i])
        overallResult['intensities_' + dimensions[i]] = overallResult['intensities'].apply(lambda b: b[i].tolist())
        
        for j in range(0, len(dimensions)):
            overallResult['influence_' + dimensions[i] + dimensions[j]] = overallResult['adjacency'].apply(lambda a: a.tolist()[i][j])
    
    overallResult.drop(['baseline', 'adjacency', 'intensities'], axis = 1, inplace = True)
    overallResult['intensity_times'] = overallResult['intensity_times'].apply(lambda b: b.tolist())
    
    return overallResult

In [ ]:
dataset = splitResultsExpKern(dataset, timestampColumns)
dataset

In [ ]:
def alignIntensityToTimestamps(df, dim):
    return dict(zip(df['intensity_times'], df['intensities_' + dim]))

for dim in timestampColumns:
    dataset['intensities_' + dim] = dataset.apply(lambda df: alignIntensityToTimestamps(df, dim), axis = 1)
    
dataset

In [ ]:
def fixTimestamps(df, column):
    result = {}
    for h in df[column].keys():
        if math.floor(h) not in result.keys():
            result[math.floor(h)] = 0
        result[math.floor(h)] = result[math.floor(h)] + df[column][h]
    return result

for dim in timestampColumns:
    dataset['intensities_' + dim] = dataset.apply(lambda df: fixTimestamps(df, 'intensities_' + dim), axis = 1)

dataset

In [ ]:
def completeTimestamps(df, colName):
    result = {}
    for t in range(0, math.floor(dayAfterEndDate) + 1):
        if t not in df[colName].keys():
            result[t] = 0
        else:
            result[t] = df[colName][t]
    return result

for colName in ['actual_intensities_', 'intensities_']:
    for dim in timestampColumns:
        dataset[colName + dim] = dataset.apply(lambda df: completeTimestamps(df, colName + dim), axis = 1)

dataset

In [ ]:
modalities = dataset['Modality'].unique()

In [ ]:
def combineIntensityDictionaries(x, y):
    combinedDictionary = {}
    
    for key in x.keys():
        if key not in combinedDictionary.keys():
            combinedDictionary[key] = 0
        combinedDictionary[key] = combinedDictionary[key] + x[key]
    for key in y.keys():
        if key not in combinedDictionary.keys():
            combinedDictionary[key] = 0
        combinedDictionary[key] = combinedDictionary[key] + y[key]
    
    return combinedDictionary
    

def sumIntensities(series):
    return reduce(combineIntensityDictionaries, series)

intensityColumns = {}
for dimension in timestampColumns:
    intensityColumns['actual_intensities_' + dimension] = sumIntensities
    intensityColumns['intensities_' + dimension] = sumIntensities

intensityPlotDataset = dataset.groupby(['Modality']).agg(intensityColumns).reset_index()
intensityPlotDataset

In [ ]:
courseStudentCount = dataset.groupby(['Modality'])['course_code'].count().to_frame('total').reset_index()
courseStudentCount

In [ ]:
intensityPlotDataset = intensityPlotDataset.merge(courseStudentCount, on=['Modality'])
intensityPlotDataset

In [ ]:
def getAverageIntensities(df, column):
    result = {}
    for k in sorted([key for key in df[column].keys()]):
        result[k] = df[column][k] / df['total']
    return result

for column in ['actual_intensities_' + dimension for dimension in timestampColumns] + ['intensities_' + dimension for dimension in timestampColumns]:
    intensityPlotDataset[column] = intensityPlotDataset.apply(lambda df: getAverageIntensities(df, column), axis = 1)

intensityPlotDataset

In [ ]:
for modality in modalities:
    fig, axs = plt.subplots(len(timestampColumns), 1)
    intensityPlotDatasetModality = intensityPlotDataset.loc[intensityPlotDataset['Modality'] == modality]
    
    for i in range(0, len(timestampColumns)):
        axs[i].plot([t for t in intensityPlotDatasetModality.iloc[0]['actual_intensities_' + timestampColumns[i]].keys()], [v for v in intensityPlotDatasetModality.iloc[0]['actual_intensities_' + timestampColumns[i]].values()], color='tab:blue')
        axs[i].plot([t for t in intensityPlotDatasetModality.iloc[0]['intensities_' + timestampColumns[i]].keys()], [v for v in intensityPlotDatasetModality.iloc[0]['intensities_' + timestampColumns[i]].values()], color='tab:orange')
        axs[i].set_title(timestampColumns[i])
    
    for ax in axs.flat:
        ax.set(xlabel='Time', ylabel='Intensity')
    
    fig.legend(handles=[Line2D([0], [0], color='tab:blue', label='Actual Intensities'), Line2D([0], [0], color='tab:orange', label='Est. Intensities')], loc='lower center')
    fig.tight_layout(pad=1.0)
    fig.set_figwidth(10)
    fig.set_figheight(10)
    fig.savefig(modality + '_Intensities_Multivariate.png')
    plt.close()

In [ ]:
for dim in timestampColumns:
    dataset['rmse_' + dim] = dataset.apply(lambda df: mean_squared_error([v for v in df['actual_intensities_' + dim].values()], [v for v in df['intensities_' + dim].values()], squared=False), axis = 1)

dataset

In [ ]:
averageAgg = {}
for dim in timestampColumns:
    averageAgg['rmse_' + dim] = np.mean

rmseModalities = dataset.groupby(['Modality']).agg(averageAgg).reset_index()
rmseModalities

In [ ]:
'''
for modality in modalities:
    fig, axs = plt.subplots(len(timestampColumns), 1)
        
    hawkesProcessResult_B_subset = dataset.loc[(dataset['Modality'] == modality)]
    for i in range(0, len(timestampColumns)):
        axs[i].scatter(hawkesProcessResult_B_subset['est_course_grade'], hawkesProcessResult_B_subset['baseline_' + timestampColumns[i]], s = 3)
        axs[i].set_title(timestampColumns[i])
    
    for ax in axs.flat:
        ax.set(xlabel='Est. Course Grade', ylabel='Base Intensity')

    fig.tight_layout(pad=1.0)
    fig.set_figwidth(10)
    fig.set_figheight(10)
    fig.savefig(modality + '_B.png')
    plt.close()
'''
rmseModalities['model'] = 'Multivariate Hawkes'
rmseModalities

In [ ]:
'''
low_q = dataset.groupby(['Modality']).agg({'est_course_grade': lambda series: np.quantile(series, 0.25)}).reset_index()
low_q = low_q.rename(columns = {'est_course_grade': 'low_q'})
low_q
'''
rmseModalitiesUnivariate = pd.read_csv(str(input()))
rmseModalitiesUnivariate['model'] = 'Univariate Hawkes'
rmseModalities = pd.concat([rmseModalities, rmseModalitiesUnivariate])
rmseModalities

In [ ]:
'''
high_q = dataset.groupby(['Modality']).agg({'est_course_grade': lambda series: np.quantile(series, 0.75)}).reset_index()
high_q = high_q.rename(columns = {'est_course_grade': 'high_q'})
high_q
'''
rmseModalitiesPoisson = pd.read_csv(str(input()))
rmseModalitiesPoisson['model'] = 'Poisson'
rmseModalities = pd.concat([rmseModalities, rmseModalitiesPoisson])
rmseModalities

In [ ]:
'''
dataset = dataset.merge(low_q, on=['Modality'])
dataset = dataset.merge(high_q, on=['Modality'])
dataset['is_low_q'] = dataset.apply(lambda df: df['est_course_grade'] < df['low_q'], axis = 1)
dataset['is_high_q'] = dataset.apply(lambda df: df['est_course_grade'] >= df['high_q'], axis = 1)
dataset
'''
for modality in modalities:
    rmseModalitiesSubset = rmseModalities.loc[rmseModalities['Modality'] == modality]
    rmseModalitiesSubset.drop(['Modality'], axis = 1, inplace = True)
    rmseModalitiesSubset = rmseModalitiesSubset.melt(id_vars = ['model'], var_name = 'dimension', value_name = 'rmse')
    rmseModalitiesSubset['dimension'] = rmseModalitiesSubset['dimension'].apply(lambda x: x.split("_")[-1])
    
    dimensions = sorted(timestampColumns)
    x = np.arange(len(dimensions))
    width = 0.25
    fig, ax = plt.subplots()
    
    models = rmseModalitiesSubset['model'].unique()
    for i in range(0, len(models)):
        offset = width * i
        rmseModalitiesSubsetModel = rmseModalitiesSubset.loc[rmseModalitiesSubset['model'] == models[i]].sort_values(by='dimension')
        rects = ax.bar(x + offset, rmseModalitiesSubsetModel['rmse'], width, label=models[i])
    
    ax.set_ylabel('Root Mean Squared Error')
    ax.set_xlabel('Multidimensional Hawkes Dimensions')
    ax.set_xticks(x + width, dimensions)
    fig.legend(loc='lower center', ncols=3)
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)
    fig.set_figwidth(10)
    fig.set_figheight(10)
    fig.savefig('GoodnessOfFit_' + modality + '.png')
    plt.close()

In [ ]:
'''
for modality in modalities:
    fig, axs = plt.subplots(len(timestampColumns), 1)
    
    hawkesProcessResult_B_subset = dataset.loc[dataset['Modality'] == modality]
    for i in range(0, len(timestampColumns)):
        sns.histplot(ax=axs[i], data=hawkesProcessResult_B_subset.loc[hawkesProcessResult_B_subset['is_high_q'] == True]['baseline_' + timestampColumns[i]], label='high', kde=True)
        sns.histplot(ax=axs[i], data=hawkesProcessResult_B_subset.loc[hawkesProcessResult_B_subset['is_low_q'] == True]['baseline_' + timestampColumns[i]], label='low', kde=True)
        axs[i].set_title(timestampColumns[i])
            
    for ax in axs.flat:
        ax.set(xlabel='Base Intensity')

    handles, labels = axs.flat[-1].get_legend_handles_labels()
    fig.legend(handles, labels, loc='lower center')
    fig.tight_layout(pad=1.0)
    fig.set_figwidth(10)
    fig.set_figheight(10)
    fig.savefig(modality + '_B_histplot.png')
    plt.close()
'''

In [ ]:
'''
for modality in modalities:
    fig, axs = plt.subplots(len(timestampColumns), len(timestampColumns))
    
    hawkesProcessResult_A_subset = dataset.loc[dataset['Modality'] == modality]
        
    for i in range(0, len(timestampColumns)):
        for j in range(0, len(timestampColumns)):
            axs[i, j].scatter(hawkesProcessResult_A_subset['est_course_grade'], hawkesProcessResult_A_subset['adjacency_' + timestampColumns[i] + "_" + timestampColumns[j]])
            axs[i, j].set_ylabel(timestampColumns[i] + "-" + timestampColumns[j])
    
    for ax in axs.flat:
        ax.set(xlabel='Est. Course Grade')

    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)
    fig.set_figwidth(30)
    fig.set_figheight(30)
    fig.savefig(modality + '_A.png')
    plt.close()
'''

In [ ]:
'''
for modality in modalities:
    fig, axs = plt.subplots(len(timestampColumns), len(timestampColumns))
    
    hawkesProcessResult_A_subset = dataset.loc[dataset['Modality'] == modality]
        
    for i in range(0, len(timestampColumns)):
        for j in range(0, len(timestampColumns)):
            sns.kdeplot(ax=axs[i, j], data=hawkesProcessResult_A_subset.loc[hawkesProcessResult_A_subset['is_high_q'] == True]['adjacency_' + timestampColumns[i] + "_" + timestampColumns[j]], label='high')
            sns.rugplot(ax=axs[i, j], data=hawkesProcessResult_A_subset.loc[hawkesProcessResult_A_subset['is_high_q'] == True]['adjacency_' + timestampColumns[i] + "_" + timestampColumns[j]], label='high')
                
            sns.kdeplot(ax=axs[i, j], data=hawkesProcessResult_A_subset.loc[hawkesProcessResult_A_subset['is_low_q'] == True]['adjacency_' + timestampColumns[i] + "_" + timestampColumns[j]], label='low')
            sns.rugplot(ax=axs[i, j], data=hawkesProcessResult_A_subset.loc[hawkesProcessResult_A_subset['is_low_q'] == True]['adjacency_' + timestampColumns[i] + "_" + timestampColumns[j]], label='low')
            axs[i, j].set_xlabel(timestampColumns[i] + "-" + timestampColumns[j])
            
    handles, labels = axs.flat[-1].get_legend_handles_labels()
    fig.legend(handles, labels, loc='lower center')
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)
    fig.set_figwidth(30)
    fig.set_figheight(30)
    fig.savefig(modality + '_A_histplot.png')
    plt.close()
'''

In [ ]:
def adjacencyColumns():
    result = []
    for i in timestampColumns:
        for j in timestampColumns:
            result.append('influence_' + i + j)
    return result

In [ ]:
'''
hawkesProcessResultRank = pd.DataFrame()
for modality in modalities:
    hawkesProcessResultSubset = dataset.loc[dataset['Modality'] == modality]
    for column in ['baseline_' + dim for dim in timestampColumns] + adjacencyColumns() + ['decay']:
        hawkesProcessResultSubset[column + "_rank"] = hawkesProcessResultSubset[column].rank(ascending=False)
    hawkesProcessResultRank = hawkesProcessResultRank.append(hawkesProcessResultSubset)

hawkesProcessResultRank
'''

In [ ]:
hawkesProcessSpearmanCorrelation = pd.DataFrame()

for modality in modalities:
    hawkesProcessResultSubset = dataset.loc[dataset['Modality'] == modality]
    for columnY in ['baseline_' + dim for dim in timestampColumns] + adjacencyColumns() + ['decay']:
        for columnX in ['baseline_' + dim for dim in timestampColumns] + adjacencyColumns() + ['decay']:
            spearmanCorrelationResult = stats.spearmanr(hawkesProcessResultSubset[columnX], hawkesProcessResultSubset[columnY])
            hawkesProcessSpearmanCorrelation = hawkesProcessSpearmanCorrelation.append(pd.DataFrame.from_dict([{'Modality': modality, 'X': columnX, 'Y': columnY, 'coefficient': spearmanCorrelationResult.correlation, 'p': spearmanCorrelationResult.pvalue}]), ignore_index = True)

hawkesProcessSpearmanCorrelation

In [ ]:
for modality in modalities:
    fig, axs = plt.subplots(2, 1)
        
    hawkesProcessResultSubset = hawkesProcessSpearmanCorrelation.loc[hawkesProcessSpearmanCorrelation['Modality'] == modality]
    hawkesProcessResultSubset.drop(['Modality'], axis = 1, inplace = True)
            
    hawkesProcessResultSubsetCoeff = hawkesProcessResultSubset[['X', 'Y', 'coefficient']]
    maxCoefficient = max(abs(min(hawkesProcessResultSubsetCoeff['coefficient'])), abs(max(hawkesProcessResultSubsetCoeff['coefficient'])))
    hawkesProcessResultSubsetCoeff = hawkesProcessResultSubsetCoeff.pivot(index='Y', columns='X', values='coefficient')
    sns.heatmap(hawkesProcessResultSubsetCoeff, ax=axs[0], annot=True, fmt=".2f", xticklabels=True, yticklabels=True, cmap=sns.color_palette("coolwarm", as_cmap=True), vmin=-maxCoefficient, vmax=maxCoefficient, square=True, linewidths=2, linecolor='w')
    axs[0].set_title("Coefficient")
    axs[0].set_xlabel("")
    axs[0].set_ylabel("")
            
    hawkesProcessResultSubsetP = hawkesProcessResultSubset[['X', 'Y', 'p']]
    hawkesProcessResultSubsetP = hawkesProcessResultSubsetP.pivot(index='Y', columns='X', values='p')
    sns.heatmap(hawkesProcessResultSubsetP, ax=axs[1], annot=True, fmt=".2f", xticklabels=True, yticklabels=True, cmap=sns.light_palette("seagreen", reverse=True, as_cmap=True), square=True, linewidths=2, linecolor='w')
    axs[1].set_title("P-Value")
    axs[1].set_xlabel("")
    axs[1].set_ylabel("")
        
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)
    fig.set_figwidth(30)
    fig.set_figheight(30)
    fig.savefig(modality + '_spearmans_corr.png')
    plt.close()

In [ ]:
hawkesProcessSpearmanCorrelation['XY'] = hawkesProcessSpearmanCorrelation.apply(lambda df: df['X'] + '-' + df['Y'] if df['X'] < df['Y'] else df['Y'] + '-' + df['X'], axis = 1)
hawkesProcessSpearmanCorrelation

In [ ]:
hawkesProcessSpearmanCorrelationTopCorrs = hawkesProcessSpearmanCorrelation.drop_duplicates(subset=['Modality', 'XY'])
hawkesProcessSpearmanCorrelationTopCorrs = hawkesProcessSpearmanCorrelationTopCorrs.loc[(hawkesProcessSpearmanCorrelation['p'] <= 0.05) & (hawkesProcessSpearmanCorrelation['X'].isin(adjacencyColumns())) & (hawkesProcessSpearmanCorrelation['Y'].isin(adjacencyColumns()))]
hawkesProcessSpearmanCorrelationTopCorrs['is_self'] = hawkesProcessSpearmanCorrelationTopCorrs.apply(lambda df: df['X'] == df['Y'], axis = 1)
hawkesProcessSpearmanCorrelationTopCorrs = hawkesProcessSpearmanCorrelationTopCorrs.loc[hawkesProcessSpearmanCorrelationTopCorrs['is_self'] == False]
hawkesProcessSpearmanCorrelationTopCorrs.drop(['is_self', 'X', 'Y'], axis = 1, inplace = True)

for column in ['coefficient', 'p']:
    hawkesProcessSpearmanCorrelationTopCorrs[column] = hawkesProcessSpearmanCorrelationTopCorrs[column].apply(lambda x: round(x, 2))

hawkesProcessSpearmanCorrelationTopCorrs

In [ ]:
hawkesProcessSpearmanCorrelationTopCorrs.loc[hawkesProcessSpearmanCorrelationTopCorrs['Modality'] == '(FULLY ONLINE)'].sort_values('coefficient', ascending=False).head(3).to_csv("MultidimensionalHawkesParameterCorr_FullyOnline_Top3_Positive.csv", header = True, index = False)

In [ ]:
hawkesProcessSpearmanCorrelationTopCorrs.loc[hawkesProcessSpearmanCorrelationTopCorrs['Modality'] == '(FULLY ONLINE)'].sort_values('coefficient', ascending=True).head(3).to_csv("MultidimensionalHawkesParameterCorr_FullyOnline_Top3_Negative.csv", header = True, index = False)

In [ ]:
hawkesProcessSpearmanCorrelationTopCorrs.loc[hawkesProcessSpearmanCorrelationTopCorrs['Modality'] == '(HYBRID)'].sort_values('coefficient', ascending=False).head(3).to_csv("MultidimensionalHawkesParameterCorr_Hybrid_Top3_Positive.csv", header = True, index = False)

In [ ]:
hawkesProcessSpearmanCorrelationTopCorrs.loc[hawkesProcessSpearmanCorrelationTopCorrs['Modality'] == '(HYBRID)'].sort_values('coefficient', ascending=True).head(3).to_csv("MultidimensionalHawkesParameterCorr_Hybrid_Top3_Negative.csv", header = True, index = False)

In [ ]:
for modality in modalities:
    fig, axs = plt.subplots(len(timestampColumns), 1)
    
    hawkesProcessResult_A_subset = dataset.loc[dataset['Modality'] == modality]
        
    for i in range(0, len(timestampColumns)):
        sns.kdeplot(ax=axs[i], data=hawkesProcessResult_A_subset['influence_' + timestampColumns[i] + timestampColumns[i]])
        axs[i].set_xlabel('influence_' + timestampColumns[i])
            
    handles, labels = axs.flat[-1].get_legend_handles_labels()
    fig.legend(handles, labels, loc='lower center')
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)
    fig.set_figwidth(30)
    fig.set_figheight(30)
    fig.savefig(modality + '_influence_density.png')
    plt.close()

In [ ]:
clusterCourseStudents = dataset[['metadata_context_id', 'metadata_user_id', 'Modality', 'course_code'] + ['influence_' + dim + dim for dim in timestampColumns]]
clusterCourseStudents

In [ ]:
clusterCourseStudentsResults = pd.DataFrame()

for modality in modalities:
    clusterCourseStudentsModality = clusterCourseStudents.loc[clusterCourseStudents['Modality'] == modality]
    clusterBy = np.array(list(zip(clusterCourseStudentsModality['influence_AA'], clusterCourseStudentsModality['influence_CC'], clusterCourseStudentsModality['influence_LL'])))
    
    learner = KMeans(n_clusters = 2, max_iter = 1000).fit(clusterBy)
    clusterCourseStudentsModality['cluster'] = learner.labels_.tolist()
    clusterCourseStudentsResults = clusterCourseStudentsResults.append(clusterCourseStudentsModality, ignore_index = True)

clusterCourseStudentsResults

In [ ]:
dataset = dataset.merge(clusterCourseStudentsResults[['metadata_context_id', 'metadata_user_id', 'Modality', 'course_code', 'cluster']], on=['metadata_context_id', 'metadata_user_id', 'Modality', 'course_code'])
dataset

In [ ]:
def getAggregatedInfluence(df, iDim):
    aggInfluence = 0
    for jDim in timestampColumns:
        aggInfluence = aggInfluence + df['influence_' + iDim + jDim]
    return aggInfluence / len(timestampColumns)

for iDim in timestampColumns:
    dataset['agg_influence_' + iDim] = dataset.apply(lambda df: getAggregatedInfluence(df, iDim), axis = 1)

dataset

In [ ]:
columnsForConversionToList = {}
for iDim in timestampColumns:
    columnsForConversionToList['baseline_' + iDim] = list
    columnsForConversionToList['agg_influence_' + iDim] = list
    for jDim in timestampColumns:
        columnsForConversionToList['influence_' + iDim + jDim] = list
columnsForConversionToList['decay'] = list

clusterHawkesParameterDiff = dataset.groupby(['Modality', 'cluster']).agg(columnsForConversionToList).reset_index()
clusterHawkesParameterDiff

In [ ]:
clusterHawkesParameterDiffResults = pd.DataFrame()

for modality in modalities:
    clusterHawkesParameterDiffModality = clusterHawkesParameterDiff.loc[clusterHawkesParameterDiff['Modality'] == modality]
    clusterHawkesParameterDiffModality.drop(['Modality'], axis = 1, inplace = True)
    
    clusterHawkesParameterDiffModality = clusterHawkesParameterDiffModality.set_index('cluster').transpose().reset_index()
    clusterHawkesParameterDiffModality['kruskal_stats'] = clusterHawkesParameterDiffModality.apply(lambda df: stats.kruskal(df[0], df[1]).statistic, axis = 1)
    clusterHawkesParameterDiffModality['kruskal_pval'] = clusterHawkesParameterDiffModality.apply(lambda df: stats.kruskal(df[0], df[1]).pvalue, axis = 1)
    clusterHawkesParameterDiffModality['Modality'] = modality
    clusterHawkesParameterDiffResults = clusterHawkesParameterDiffResults.append(clusterHawkesParameterDiffModality, ignore_index = True)
    
clusterHawkesParameterDiffResults

In [ ]:
clusterHawkesParameterDiffResults.drop([0, 1], axis = 1, inplace = True)
clusterHawkesParameterDiffResults['is_significant'] = clusterHawkesParameterDiffResults['kruskal_pval'].apply(lambda x: x <= 0.001)
clusterHawkesParameterDiffResults

In [ ]:
clusterHawkesParameterDiffResults['color'] = clusterHawkesParameterDiffResults['is_significant'].apply(lambda x: 'tab:blue' if x else 'tab:gray')
clusterHawkesParameterDiffResults

In [ ]:
for modality in modalities:
    clusterHawkesParameterDiffResultsModality = clusterHawkesParameterDiffResults.loc[(clusterHawkesParameterDiffResults['Modality'] == modality) & (~(clusterHawkesParameterDiffResults['index'] == 'decay'))].sort_values(by=['kruskal_stats'])
    clusterHawkesParameterDiffResultsModality.drop(['Modality'], axis = 1, inplace = True)
    
    fig, ax = plt.subplots()
    ax.barh(clusterHawkesParameterDiffResultsModality['index'], clusterHawkesParameterDiffResultsModality['kruskal_stats'], color=clusterHawkesParameterDiffResultsModality['color'])
    ax.set_xlabel('Kruskal-Wallis Coefficient')
    ax.set_ylabel('Multidimensional Hawkes Parameters')
    fig.legend(handles=[Line2D([0], [0], color='tab:blue', label='P-Value <= 0.001'), Line2D([0], [0], color='tab:gray', label='P-Value > 0.001')], loc='lower center')
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)
    fig.set_figwidth(50)
    fig.set_figheight(10)
    fig.savefig('ClusterHawkesParameterDiffResults_' + modality + '.png')
    plt.close()

In [ ]:
#clusterHawkesParameterDiffResults.drop(['kruskal_pval'], axis = 1).pivot(index = 'Modality', columns = 'index', values = 'kruskal_stats').reset_index().to_csv("MultivariateHawkes_Kruskal_Results_Stats.csv", header = True, index = False)

In [ ]:
#clusterHawkesParameterDiffResults.drop(['kruskal_stats'], axis = 1).pivot(index = 'Modality', columns = 'index', values = 'kruskal_pval').reset_index().to_csv("MultivariateHawkes_Kruskal_Results_PValue.csv", header = True, index = False)

In [ ]:
columnsForAveragingList = {}
for iDim in timestampColumns:
    columnsForAveragingList['baseline_' + iDim] = np.mean
    columnsForAveragingList['agg_influence_' + iDim] = np.mean
    for jDim in timestampColumns:
        columnsForAveragingList['influence_' + iDim + jDim] = np.mean
columnsForAveragingList['decay'] = np.mean

clusterHawkesParameterAve = dataset.groupby(['Modality', 'cluster']).agg(columnsForAveragingList).reset_index()
clusterHawkesParameterAve

In [ ]:
clusterHawkesParameterDiffResultsFullyOnline = clusterHawkesParameterDiffResults.loc[clusterHawkesParameterDiffResults['Modality'] == '(FULLY ONLINE)'][['index', 'is_significant']]
clusterHawkesParameterDiffResultsHybrid = clusterHawkesParameterDiffResults.loc[clusterHawkesParameterDiffResults['Modality'] == '(HYBRID)'][['index', 'is_significant']]
clusterHawkesParameterDiffResultsSignificance = clusterHawkesParameterDiffResultsFullyOnline.merge(clusterHawkesParameterDiffResultsHybrid, on=['index'])
clusterHawkesParameterDiffResultsSignificance

In [ ]:
def comparePValuesModalities(df):
    return df['is_significant_x'] and df['is_significant_y']

clusterHawkesParameterDiffResultsSignificance['both_significant'] = clusterHawkesParameterDiffResultsSignificance.apply(comparePValuesModalities, axis = 1)
clusterHawkesParameterDiffResultsSignificance

In [ ]:
clusterHawkesParameterDiffResultsSignificance = clusterHawkesParameterDiffResultsSignificance.loc[~(clusterHawkesParameterDiffResultsSignificance['index'] == 'decay')]
clusterHawkesParameterDiffResultsSignificance = clusterHawkesParameterDiffResultsSignificance[['index', 'both_significant']]
clusterHawkesParameterDiffResultsSignificance

In [ ]:
def fixColorPerCluster(df):
    if(df['both_significant'] == False):
        if(df['cluster'] == 0):
            return 'dimgray'
        else:
            return 'silver'
    else:
        if(df['cluster'] == 0):
            return 'tab:blue'
        else:
            return 'tab:orange'

for modality in modalities:
    clusterHawkesParameterAveModality = clusterHawkesParameterAve.loc[clusterHawkesParameterAve['Modality'] == modality]
    clusterHawkesParameterAveModality.drop(['Modality', 'decay'], axis = 1, inplace = True)
    clusterHawkesParameterAveModality = clusterHawkesParameterAveModality.melt(id_vars='cluster', var_name='index', value_name='ave').reset_index()
    
    clusterHawkesParameterAveModality = clusterHawkesParameterAveModality.merge(clusterHawkesParameterDiffResultsSignificance, on=['index'])
    clusterHawkesParameterAveModality['color'] = clusterHawkesParameterAveModality.apply(fixColorPerCluster, axis = 1)
    
    parameters = sorted(['baseline_' + dim for dim in timestampColumns] + adjacencyColumns() + ['agg_influence_' + dim for dim in timestampColumns], reverse = True)
    x = np.arange(len(parameters))
    width = 0.25
    fig, ax = plt.subplots()
    
    for cluster in [0, 1]:
        offset = width * cluster
        clusterHawkesParameterAveModalityCluster = clusterHawkesParameterAveModality.loc[clusterHawkesParameterAveModality['cluster'] == cluster].sort_values(by='index', ascending = False)
        rects = ax.barh(x + offset, clusterHawkesParameterAveModalityCluster['ave'], width, color=clusterHawkesParameterAveModalityCluster['color'])
    
    ax.set_xlabel('Average Across Course-Student Pairs')
    ax.set_ylabel('Multidimensional Hawkes Parameters')
    ax.set_yticks(x + (width / 2), parameters)
    fig.legend(handles=[Line2D([0], [0], color='tab:blue', label='Cluster 0, P-Value <= 0.001 for both Fully Online and Hybrid Classes'), Line2D([0], [0], color='tab:orange', label='Cluster 1, P-Value <= 0.001 for both Fully Online and Hybrid Classes'), Line2D([0], [0], color='dimgray', label='Cluster 0, P-Value > 0.001 for either Fully Online or Hybrid Classes'), Line2D([0], [0], color='silver', label='Cluster 1, P-Value > 0.001 for either Fully Online or Hybrid Classes')], ncols=4, loc='lower center')
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)
    fig.set_figwidth(50)
    fig.set_figheight(10)
    fig.savefig('ClusterHawkesParameterAve_' + modality + '.png')
    plt.close()

In [ ]:
#clusterHawkesParameterAve.to_csv("MultivariateHawkes_Metrics_Average_Groups.csv", header = True, index = False)

In [ ]:
dataset = dataset.rename(columns={"ave_delay_graded": "ave_delay_A", "ave_delay_ungraded": "ave_delay_C"})
dataset

In [ ]:
delayPerClusterDiff = pd.DataFrame()

for modality in modalities:
    subDataset = dataset.loc[dataset['Modality'] == modality]
    
    for dim in timestampColumns:
        result = {}
        subDatasetDim = subDataset[['cluster', 'ave_delay_' + dim]]
        subDatasetDim = subDatasetDim.dropna(subset=['ave_delay_' + dim])
        
        result['Modality'] = modality
        result['dimension'] = dim
        result['kruskal_stats'] = stats.kruskal(subDatasetDim.loc[subDatasetDim['cluster'] == 0]['ave_delay_' + dim], subDatasetDim.loc[subDatasetDim['cluster'] == 1]['ave_delay_' + dim]).statistic
        result['kruskal_pval'] = stats.kruskal(subDatasetDim.loc[subDatasetDim['cluster'] == 0]['ave_delay_' + dim], subDatasetDim.loc[subDatasetDim['cluster'] == 1]['ave_delay_' + dim]).pvalue
        delayPerClusterDiff = delayPerClusterDiff.append(result, ignore_index = True)

delayPerClusterDiff

In [ ]:
delayPerClusterDiff['is_significant'] = delayPerClusterDiff['kruskal_pval'].apply(lambda x: x <= 0.001)
delayPerClusterDiff

In [ ]:
delayPerClusterDiff['color'] = delayPerClusterDiff['is_significant'].apply(lambda x: 'tab:blue' if x else 'tab:gray')
delayPerClusterDiff

In [ ]:
for modality in modalities:
    delayPerClusterDiffModality = delayPerClusterDiff.loc[delayPerClusterDiff['Modality'] == modality]
    delayPerClusterDiffModality.drop(['Modality'], axis = 1, inplace = True)
    
    fig, ax = plt.subplots()
    ax.bar(delayPerClusterDiffModality['dimension'], delayPerClusterDiffModality['kruskal_stats'], color=delayPerClusterDiffModality['color'])
    ax.set_ylabel('Kruskal-Wallis Coefficient')
    ax.set_xlabel('Multidimensional Hawkes Dimensions')
    fig.legend(handles=[Line2D([0], [0], color='tab:blue', label='P-Value <= 0.001'), Line2D([0], [0], color='tab:gray', label='P-Value > 0.001')], loc='lower center')
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)
    fig.set_figwidth(10)
    fig.set_figheight(10)
    fig.savefig('DelayPerClusterDiff_' + modality + '.png')
    plt.close()

In [ ]:
averageDelayPerCluster = pd.DataFrame()

for modality in modalities:
    for cluster in dataset['cluster'].unique():
        subDataset = dataset.loc[(dataset['Modality'] == modality) & (dataset['cluster'] == cluster)]
        for dim in timestampColumns:
            result = {'Modality': modality, 'cluster': cluster, 'dimension': dim}
            subDataset = subDataset.dropna(subset=['ave_delay_' + dim])
            result['ave_delay'] = np.mean(subDataset['ave_delay_' + dim])
            averageDelayPerCluster = averageDelayPerCluster.append(result, ignore_index = True)

averageDelayPerCluster

In [ ]:
delayPerClusterDiffSignificance = delayPerClusterDiff[['Modality', 'dimension', 'is_significant']]
delayPerClusterDiffSignificance

In [ ]:
averageDelayPerCluster = averageDelayPerCluster.merge(delayPerClusterDiffSignificance, on=['Modality', 'dimension'])
averageDelayPerCluster

In [ ]:
for modality in modalities:
    averageDelayPerClusterModality = averageDelayPerCluster.loc[averageDelayPerCluster['Modality'] == modality]
    averageDelayPerClusterModality.drop(['Modality'], axis = 1, inplace = True)
    averageDelayPerClusterModality = averageDelayPerClusterModality.rename(columns={'is_significant': 'both_significant'})
    averageDelayPerClusterModality['color'] = averageDelayPerClusterModality.apply(fixColorPerCluster, axis = 1)
    
    dimensions = sorted(timestampColumns)
    x = np.arange(len(dimensions))
    width = 0.25
    fig, ax = plt.subplots()
    
    for cluster in [0, 1]:
        offset = width * cluster
        averageDelayPerClusterModalityCluster = averageDelayPerClusterModality.loc[averageDelayPerClusterModality['cluster'] == cluster].sort_values(by='dimension')
        rects = ax.bar(x + offset, averageDelayPerClusterModalityCluster['ave_delay'], width, color=averageDelayPerClusterModalityCluster['color'])
    
    ax.set_ylabel('Average Delay Across Course-Student Pairs')
    ax.set_xlabel('Multidimensional Hawkes Dimensions')
    ax.set_xticks(x + (width / 2), dimensions)
    fig.legend(handles=[Line2D([0], [0], color='tab:blue', label='Cluster 0, P-Value <= 0.001'), Line2D([0], [0], color='tab:orange', label='Cluster 1, P-Value <= 0.001'), Line2D([0], [0], color='dimgray', label='Cluster 0, P-Value > 0.001'), Line2D([0], [0], color='silver', label='Cluster 1, P-Value > 0.001')], loc='lower center')
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)
    fig.set_figwidth(10)
    fig.set_figheight(15)
    fig.savefig('AverageDelayPerCluster_' + modality + '.png')
    plt.close()

In [ ]:
def adjustedDelay(df, currentDim):
    influenceBias = 0
    for dim in timestampColumns:
        if not (currentDim == dim):
            influenceBias = influenceBias + (1 / (1 - df['influence_' + currentDim + dim]))
    
    return df['ave_delay_' + currentDim] + (1 / influenceBias)

for dim in timestampColumns:
    dataset['ave_delay_adj_' + dim] = dataset.apply(lambda df: adjustedDelay(df, dim), axis = 1)

dataset

In [ ]:
hawkesProcessSpearmanCorrelationGrades = pd.DataFrame()

for modality in modalities:
    hawkesProcessResultSubset = dataset.loc[dataset['Modality'] == modality]
    for dim in timestampColumns:
        spearmanCorrelationResultDelay = stats.spearmanr(hawkesProcessResultSubset['est_course_grade'], hawkesProcessResultSubset['ave_delay_' + dim])
        spearmanCorrelationResultDelayAdj = stats.spearmanr(hawkesProcessResultSubset['est_course_grade'], hawkesProcessResultSubset['ave_delay_adj_' + dim])
        hawkesProcessSpearmanCorrelationGrades = hawkesProcessSpearmanCorrelationGrades.append(pd.DataFrame.from_dict([{'Modality': modality, 'dimension': dim, 'coefficient_delay': spearmanCorrelationResultDelay.correlation, 'p_delay': spearmanCorrelationResultDelay.pvalue, 'coefficient_delay_adj': spearmanCorrelationResultDelayAdj.correlation, 'p_delay_adj': spearmanCorrelationResultDelayAdj.pvalue}]), ignore_index = True)

for suffix in ['', '_adj']:
    hawkesProcessSpearmanCorrelationGrades['coefficient_delay' + suffix] = hawkesProcessSpearmanCorrelationGrades['coefficient_delay' + suffix].apply(lambda x: round(x, 2))
    hawkesProcessSpearmanCorrelationGrades['p_delay' + suffix] = hawkesProcessSpearmanCorrelationGrades['p_delay' + suffix].apply(lambda x: round(x, 2))
        
hawkesProcessSpearmanCorrelationGrades

In [ ]:
hawkesProcessSpearmanCorrelationGrades.to_csv("MultidimensionalHawkesParametersDelayCorrGrades.csv", header = True, index = False)